<a href="https://colab.research.google.com/github/seonghaiiiii/jeju-food/blob/main/non_sunday_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import mean_squared_error, r2_score

#Data load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data load
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')
trade = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/international_trade.csv')
test = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/test.csv')

#trian 기본 정보 및 일요일 제외

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB


In [ ]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [ ]:
# 'timestamp' 열을 datetime 형식으로 변환합니다.
train['timestamp'] = pd.to_datetime(train['timestamp'])

# 'timestamp' 열에서 요일이 일요일이 아닌 행들을 선택합니다.
train = train[train['timestamp'].dt.day_name() != 'Sunday']

#데이터 자르기

In [ ]:
# ID 열의 앞 6자를 기준으로 groupby하여 데이터프레임을 분할
grouped = train.groupby(train['ID'].str[:6])

# 그룹별로 데이터프레임을 만들어 저장
dataframes = {}
for group_name, group_data in grouped:
    dataframes[group_name] = group_data

# 결과 확인
for group_name, group_df in dataframes.items():
    print(f"Group: {group_name}")
    print(group_df)


Group: BC_A_J
                    ID  timestamp item corporation location  supply(kg)  \
42644  BC_A_J_20190101 2019-01-01   BC           A        J         0.0   
42645  BC_A_J_20190102 2019-01-02   BC           A        J         0.0   
42646  BC_A_J_20190103 2019-01-03   BC           A        J      7616.0   
42647  BC_A_J_20190104 2019-01-04   BC           A        J      7488.0   
42648  BC_A_J_20190105 2019-01-05   BC           A        J     10408.0   
...                ...        ...  ...         ...      ...         ...   
44162  BC_A_J_20230227 2023-02-27   BC           A        J     11702.0   
44163  BC_A_J_20230228 2023-02-28   BC           A        J      6056.0   
44164  BC_A_J_20230301 2023-03-01   BC           A        J      5896.0   
44165  BC_A_J_20230302 2023-03-02   BC           A        J      8600.0   
44166  BC_A_J_20230303 2023-03-03   BC           A        J     10728.0   

       price(원/kg)  
42644          0.0  
42645          0.0  
42646       2559.0  
4

In [ ]:
TG_A_J = dataframes.get('TG_A_J')
TG_A_S = dataframes.get('TG_A_S')
TG_B_J = dataframes.get('TG_B_J')
TG_B_S = dataframes.get('TG_B_S')
TG_C_J = dataframes.get('TG_C_J')
TG_C_S = dataframes.get('TG_C_S')
TG_D_J = dataframes.get('TG_D_J')
TG_D_S = dataframes.get('TG_D_S')
TG_E_J = dataframes.get('TG_E_J')
TG_E_S = dataframes.get('TG_E_S')

In [ ]:
CR_A_J = dataframes.get('CR_A_J')
CR_A_S = dataframes.get('CR_A_S')
CR_B_J = dataframes.get('CR_B_J')
CR_B_S = dataframes.get('CR_B_S')
CR_C_J = dataframes.get('CR_C_J')
CR_C_S = dataframes.get('CR_C_S')
CR_D_J = dataframes.get('CR_D_J')
CR_D_S = dataframes.get('CR_D_S')
CR_E_J = dataframes.get('CR_E_J')
CR_E_S = dataframes.get('CR_E_S')

In [ ]:
CB_A_J = dataframes.get('CB_A_J')
CB_A_S = dataframes.get('CB_A_S')
CB_B_J = dataframes.get('CB_B_J')
CB_B_S = dataframes.get('CB_B_S')
CB_C_J = dataframes.get('CB_C_J')
CB_C_S = dataframes.get('CB_C_S')
CB_D_J = dataframes.get('CB_D_J')
CB_D_S = dataframes.get('CB_D_S')
CB_E_J = dataframes.get('CB_E_J')
CB_E_S = dataframes.get('CB_E_S')

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID'].unique()

# 고유한 값들 출력
for unique_id in unique_ids:
    print(unique_id)

TG_A_J_20230304
TG_A_J_20230305
TG_A_J_20230306
TG_A_J_20230307
TG_A_J_20230308
TG_A_J_20230309
TG_A_J_20230310
TG_A_J_20230311
TG_A_J_20230312
TG_A_J_20230313
TG_A_J_20230314
TG_A_J_20230315
TG_A_J_20230316
TG_A_J_20230317
TG_A_J_20230318
TG_A_J_20230319
TG_A_J_20230320
TG_A_J_20230321
TG_A_J_20230322
TG_A_J_20230323
TG_A_J_20230324
TG_A_J_20230325
TG_A_J_20230326
TG_A_J_20230327
TG_A_J_20230328
TG_A_J_20230329
TG_A_J_20230330
TG_A_J_20230331
TG_A_S_20230304
TG_A_S_20230305
TG_A_S_20230306
TG_A_S_20230307
TG_A_S_20230308
TG_A_S_20230309
TG_A_S_20230310
TG_A_S_20230311
TG_A_S_20230312
TG_A_S_20230313
TG_A_S_20230314
TG_A_S_20230315
TG_A_S_20230316
TG_A_S_20230317
TG_A_S_20230318
TG_A_S_20230319
TG_A_S_20230320
TG_A_S_20230321
TG_A_S_20230322
TG_A_S_20230323
TG_A_S_20230324
TG_A_S_20230325
TG_A_S_20230326
TG_A_S_20230327
TG_A_S_20230328
TG_A_S_20230329
TG_A_S_20230330
TG_A_S_20230331
TG_B_J_20230304
TG_B_J_20230305
TG_B_J_20230306
TG_B_J_20230307
TG_B_J_20230308
TG_B_J_20230309
TG_B_J_2

In [ ]:
#ID 앞에 6자만 필요해서 자른거임
df1['ID_1'] = df1['ID'].str[:6]

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID_1'].unique()

unique_ids

# 총 39개

array(['TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S',
       'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S', 'CR_A_J', 'CR_B_J',
       'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S', 'CB_A_J',
       'CB_A_S', 'CB_D_J', 'CB_E_J', 'RD_A_J', 'RD_A_S', 'RD_C_S',
       'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S', 'BC_A_J', 'BC_A_S',
       'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J',
       'BC_E_S', 'CB_F_J', 'RD_F_J'], dtype=object)

In [ ]:
TG_A_J = dataframes.get('TG_A_J')
TG_A_S = dataframes.get('TG_A_S')
TG_B_J = dataframes.get('TG_B_J')
TG_B_S = dataframes.get('TG_B_S')
TG_C_J = dataframes.get('TG_C_J')
TG_C_S = dataframes.get('TG_C_S')
TG_D_J = dataframes.get('TG_D_J')
TG_D_S = dataframes.get('TG_D_S')
TG_E_J = dataframes.get('TG_E_J')
TG_E_S = dataframes.get('TG_E_S')
CR_A_J = dataframes.get('CR_A_J')
CR_B_J = dataframes.get('CR_B_J')
CR_C_J = dataframes.get('CR_C_J')
CR_D_J = dataframes.get('CR_D_J')
CR_D_S = dataframes.get('CR_D_S')
CR_E_J = dataframes.get('CR_E_J')
CR_E_S = dataframes.get('CR_E_S')
CB_A_J = dataframes.get('CB_A_J')
CB_A_S = dataframes.get('CB_A_S')
CB_D_J = dataframes.get('CB_D_J')
CB_E_J = dataframes.get('CB_E_J')
RD_A_J = dataframes.get('RD_A_J')
RD_A_S = dataframes.get('RD_A_S')
RD_C_S = dataframes.get('RD_C_S')
RD_D_J = dataframes.get('RD_D_J')
RD_D_S = dataframes.get('RD_D_S')
RD_E_J = dataframes.get('RD_E_J')
RD_E_S = dataframes.get('RD_E_S')
BC_A_J = dataframes.get('BC_A_J')
BC_A_S = dataframes.get('BC_A_S')
BC_B_J = dataframes.get('BC_B_J')
BC_B_S = dataframes.get('BC_B_S')
BC_C_J = dataframes.get('BC_C_J')
BC_C_S = dataframes.get('BC_C_S')
BC_D_J = dataframes.get('BC_D_J')
BC_E_J = dataframes.get('BC_E_J')
BC_E_S = dataframes.get('BC_E_S')
CB_F_J = dataframes.get('CB_F_J')
RD_F_J = dataframes.get('RD_F_J')

In [ ]:
# 원하는 열 이름 목록을 만듭니다.
columns_to_extract = ['TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S',
                      'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S', 'CR_A_J', 'CR_B_J',
                      'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S', 'CB_A_J',
                      'CB_A_S', 'CB_D_J', 'CB_E_J', 'RD_A_J', 'RD_A_S', 'RD_C_S',
                      'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S', 'BC_A_J', 'BC_A_S',
                      'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J',
                      'BC_E_S', 'CB_F_J', 'RD_F_J']

# 열 이름 목록을 기반으로 데이터프레임에서 열을 반복적으로 추출합니다.
for column_name in columns_to_extract:
    extracted_column = dataframes.get(column_name)
    # 여기에서 extracted_column을 사용하여 원하는 작업을 수행할 수 있습니다.
    # 예를 들어, 추출된 열로 특정 계산을 수행하거나 다른 작업을 수행할 수 있습니다.


In [ ]:
# 삭제할 열 이름 리스트
columns_to_delete = ['ID', 'item', 'corporation', 'location','supply(kg)']
# 변수들에 대해 열 삭제 수행
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

for df in dataframes:
    df.drop(columns=columns_to_delete, inplace=True)


In [ ]:
# 변수들에 대해 열 이름 변경 수행
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

for df in dataframes:
    if 'price(원/kg)' in df.columns:
        df.rename(columns={'price(원/kg)': 'price'}, inplace=True)

In [ ]:
TG_A_J.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1306 entries, 0 to 1522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  1306 non-null   datetime64[ns]
 1   price      1306 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 30.6 KB


In [ ]:
TG_A_J

,timestamp,price
0,2019-01-01,0.0
1,2019-01-02,0.0
2,2019-01-03,1728.0
3,2019-01-04,1408.0
4,2019-01-05,1250.0
...,...,...
1518,2023-02-27,2657.0
1519,2023-02-28,3922.0
1520,2023-03-01,3397.0
1521,2023-03-02,3195.0


#TG_A_J 한해서 테스트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# Assuming TG_A_J is your DataFrame
# Convert timestamp to datetime and add a new column
TG_A_J['datetime'] = pd.to_datetime(TG_A_J['timestamp'])

# Extract relevant columns
data = TG_A_J[['datetime', 'price']]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data[['price']] = scaler.fit_transform(data[['price']])

# Function to prepare the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Set the look-back period (you can adjust this)
look_back = 1300

# Create the dataset
X, Y = create_dataset(data[['price']].values, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(1, look_back)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(X, Y, epochs=3, batch_size=1, verbose=2)

# Generate future timestamps from 2023-03-04 to 2023-03-31
future_dates = pd.date_range(start='2023-03-04', end='2023-03-31')

# Make predictions for the future dates
predicted_prices_normalized = []

for i in range(len(future_dates)):
    input_data = data['price'][-look_back:].values.reshape(1, 1, look_back)
    predicted_price_normalized = model.predict(input_data)[0, 0]
    predicted_prices_normalized.append(predicted_price_normalized)
    data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)

# Inverse transform the predicted prices to get the actual values
predicted_prices = scaler.inverse_transform(np.array(predicted_prices_normalized).reshape(-1, 1))

# Create a DataFrame for the predicted prices and corresponding timestamps
predicted_df = pd.DataFrame(index=future_dates, data={'predicted_price': predicted_prices.flatten()})

# Add the 'timestamp' column to predicted_df
predicted_df['timestamp'] = predicted_df.index

# Display the updated predicted_df
print(predicted_df.head())


<ipython-input-46-a4c8fb98c06f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['price']] = scaler.fit_transform(data[['price']])


Epoch 1/3
6/6 - 2s - loss: 0.0573 - 2s/epoch - 400ms/step
Epoch 2/3
6/6 - 0s - loss: 0.0987 - 41ms/epoch - 7ms/step
Epoch 3/3
6/6 - 0s - loss: 0.0422 - 38ms/epoch - 6ms/step
1/1 [==============================] - 0s 22ms/step


<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 23ms/step


<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)


...

<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 23ms/step


<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 24ms/step
            predicted_price  timestamp
2023-03-04      1811.442017 2023-03-04
2023-03-05      1743.797485 2023-03-05
2023-03-06      2068.828369 2023-03-06
2023-03-07      2175.091553 2023-03-07
2023-03-08      1977.945068 2023-03-08


<ipython-input-46-a4c8fb98c06f>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[i], 'price': predicted_price_normalized}, ignore_index=True)


In [ ]:
predicted_df

,predicted_price,timestamp
2023-03-04,1811.442017,2023-03-04
2023-03-05,1743.797485,2023-03-05
2023-03-06,2068.828369,2023-03-06
2023-03-07,2175.091553,2023-03-07
2023-03-08,1977.945068,2023-03-08
2023-03-09,1949.572144,2023-03-09
2023-03-10,1896.740356,2023-03-10
2023-03-11,1411.053711,2023-03-11
2023-03-12,2123.319824,2023-03-12
2023-03-13,1728.517822,2023-03-13


#조기종료 없는 테스트 버전

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# List of dataframes
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

# Set the look-back period
look_back = 1300

# Initialize a dictionary to store predicted DataFrames
predicted_dfs = {}

# Loop through each dataframe
for i, df in enumerate(dataframes):
    # Convert timestamp to datetime and add a new column
    df['datetime'] = pd.to_datetime(df['timestamp'])

    # Extract relevant columns
    data = df[['datetime', 'price']]

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['price']] = scaler.fit_transform(data[['price']])

    # Create the dataset
    X, Y = create_dataset(data[['price']].values, look_back)

    # Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, input_shape=(1, look_back)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Fit the model
    model.fit(X, Y, epochs=2, batch_size=1, verbose=2)

    # Generate future timestamps from 2023-03-04 to 2023-03-31
    future_dates = pd.date_range(start='2023-03-04', end='2023-03-31')

    # Make predictions for the future dates
    predicted_prices_normalized = []

    for j in range(len(future_dates)):
        input_data = data['price'][-look_back:].values.reshape(1, 1, look_back)
        predicted_price_normalized = model.predict(input_data)[0, 0]
        predicted_prices_normalized.append(predicted_price_normalized)
        data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)

    # Inverse transform the predicted prices to get the actual values
    predicted_prices = scaler.inverse_transform(np.array(predicted_prices_normalized).reshape(-1, 1))

    # Create a DataFrame for the predicted prices and corresponding timestamps
    predicted_df = pd.DataFrame(index=future_dates, data={'predicted_price': predicted_prices.flatten()})

    # Add the 'timestamp' column to predicted_df
    predicted_df['timestamp'] = predicted_df.index

    # Store the result in the dictionary
    predicted_dfs[f'predicted_df{i+1}'] = predicted_df

# Display the first few rows of the first predicted dataframe
print(predicted_dfs['predicted_df1'].head())


<ipython-input-50-a8aac1d1e2f8>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['price']] = scaler.fit_transform(data[['price']])


Epoch 1/2
6/6 - 3s - loss: 0.0514 - 3s/epoch - 479ms/step
Epoch 2/2
6/6 - 0s - loss: 0.0931 - 42ms/epoch - 7ms/step
1/1 [==============================] - 0s 22ms/step


<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 22ms/step


<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


...

<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 23ms/step


<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 24ms/step
            predicted_price  timestamp
2023-03-04      3651.327148 2023-03-04
2023-03-05      4566.991699 2023-03-05
2023-03-06      3852.527344 2023-03-06
2023-03-07      3732.767822 2023-03-07
2023-03-08      4513.167969 2023-03-08


<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-50-a8aac1d1e2f8>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


In [ ]:
predicted_dfs['predicted_df4']

,predicted_price,timestamp
2023-03-04,5141.975586,2023-03-04
2023-03-05,4691.511719,2023-03-05
2023-03-06,4714.789062,2023-03-06
2023-03-07,5002.733398,2023-03-07
2023-03-08,4969.109863,2023-03-08
2023-03-09,5037.608887,2023-03-09
2023-03-10,5134.072754,2023-03-10
2023-03-11,5226.172363,2023-03-11
2023-03-12,5125.806641,2023-03-12
2023-03-13,5043.750977,2023-03-13


#준형 이거야 아마 돌아갈거야

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime, timedelta

# Function to prepare the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# List of dataframes
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

# Set the look-back period
look_back = 1300

# Initialize a dictionary to store predicted DataFrames
predicted_dfs = {}

# Loop through each dataframe
for i, df in enumerate(dataframes):
    # Convert timestamp to datetime and add a new column
    df['datetime'] = pd.to_datetime(df['timestamp'])

    # Extract relevant columns
    data = df[['datetime', 'price']]

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['price']] = scaler.fit_transform(data[['price']])

    # Create the dataset
    X, Y = create_dataset(data[['price']].values, look_back)

    # Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, input_shape=(1, look_back)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Add early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

    # Fit the model with early stopping
    model.fit(X, Y, epochs=50, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate future timestamps from 2023-03-04 to 2023-03-31
    future_dates = pd.date_range(start='2023-03-04', end='2023-03-31')

    # Make predictions for the future dates
    predicted_prices_normalized = []

    for j in range(len(future_dates)):
        input_data = data['price'][-look_back:].values.reshape(1, 1, look_back)
        predicted_price_normalized = model.predict(input_data)[0, 0]
        predicted_prices_normalized.append(predicted_price_normalized)
        data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)

    # Inverse transform the predicted prices to get the actual values
    predicted_prices = scaler.inverse_transform(np.array(predicted_prices_normalized).reshape(-1, 1))

    # Create a DataFrame for the predicted prices and corresponding timestamps
    predicted_df = pd.DataFrame(index=future_dates, data={'predicted_price': predicted_prices.flatten()})

    # Add the 'timestamp' column to predicted_df
    predicted_df['timestamp'] = predicted_df.index

    # Store the result in the dictionary
    predicted_dfs[f'predicted_df{i+1}'] = predicted_df

# Display the first few rows of the first predicted dataframe
print(predicted_dfs['predicted_df1'].head())


<ipython-input-22-84ac338477ba>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['price']] = scaler.fit_transform(data[['price']])


Epoch 1/50
6/6 - 3s - loss: 0.0912 - 3s/epoch - 557ms/step
Epoch 2/50
6/6 - 0s - loss: 0.0560 - 50ms/epoch - 8ms/step
Epoch 3/50
6/6 - 0s - loss: 0.0161 - 42ms/epoch - 7ms/step
Epoch 4/50
6/6 - 0s - loss: 0.0050 - 41ms/epoch - 7ms/step
Epoch 5/50
6/6 - 0s - loss: 0.0063 - 41ms/epoch - 7ms/step
Epoch 6/50
6/6 - 0s - loss: 6.9523e-04 - 44ms/epoch - 7ms/step
Epoch 7/50
6/6 - 0s - loss: 0.0014 - 42ms/epoch - 7ms/step
Epoch 8/50
6/6 - 0s - loss: 5.7586e-04 - 43ms/epoch - 7ms/step
Epoch 9/50
6/6 - 0s - loss: 4.8717e-04 - 43ms/epoch - 7ms/step
Epoch 10/50
6/6 - 0s - loss: 9.8699e-05 - 42ms/epoch - 7ms/step
Epoch 11/50
6/6 - 0s - loss: 2.4085e-04 - 40ms/epoch - 7ms/step
Epoch 12/50
6/6 - 0s - loss: 8.8005e-05 - 41ms/epoch - 7ms/step
Epoch 13/50
6/6 - 0s - loss: 7.0829e-05 - 43ms/epoch - 7ms/step
Epoch 14/50
6/6 - 0s - loss: 2.6433e-05 - 41ms/epoch - 7ms/step
Epoch 15/50
6/6 - 0s - loss: 2.5695e-05 - 45ms/epoch - 8ms/step
Epoch 16/50
6/6 - 0s - loss: 6.5396e-06 - 43ms/epoch - 7ms/step
Epoch 17/

<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 35ms/step


<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


...

<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 22ms/step


<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 25ms/step
            predicted_price  timestamp
2023-03-04      3040.020996 2023-03-04
2023-03-05      3686.450195 2023-03-05
2023-03-06      3312.134521 2023-03-06
2023-03-07      3389.550049 2023-03-07
2023-03-08      3294.054443 2023-03-08


<ipython-input-22-84ac338477ba>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


In [ ]:
# Create variables for each predicted dataframe
for i, key in enumerate(predicted_dfs.keys()):
    globals()[f'predicted_df{i+1}'] = predicted_dfs[key]

In [ ]:
# Create variables for each predicted dataframe with new names
new_names = [
    'TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S', 'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S',
    'CR_A_J', 'CR_B_J', 'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S',
    'CB_A_J', 'CB_A_S', 'CB_D_J', 'CB_E_J',
    'RD_A_J', 'RD_A_S', 'RD_C_S', 'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S',
    'BC_A_J', 'BC_A_S', 'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J', 'BC_E_S',
    'CB_F_J', 'RD_F_J'
]

for i, key in enumerate(predicted_dfs.keys()):
    globals()[f'pred_{new_names[i]}'] = predicted_dfs[key]


In [ ]:
import pandas as pd

# Assuming df1 is your DataFrame
# Convert the ID column to string
df1['ID_2'] = df1['ID'].astype(str).str.extract(r'_(\d{8})')

# Convert the extracted date to datetime format
df1['ID_2'] = pd.to_datetime(df1['ID_2'], format='%Y%m%d')

# Display the updated DataFrame
df1

,ID,answer,ID_1,ID_2
0,TG_A_J_20230304,0,TG_A_J,2023-03-04
1,TG_A_J_20230305,0,TG_A_J,2023-03-05
2,TG_A_J_20230306,0,TG_A_J,2023-03-06
3,TG_A_J_20230307,0,TG_A_J,2023-03-07
4,TG_A_J_20230308,0,TG_A_J,2023-03-08
...,...,...,...,...
1087,RD_F_J_20230327,0,RD_F_J,2023-03-27
1088,RD_F_J_20230328,0,RD_F_J,2023-03-28
1089,RD_F_J_20230329,0,RD_F_J,2023-03-29
1090,RD_F_J_20230330,0,RD_F_J,2023-03-30


In [ ]:
pred_TG_A_J

,predicted_price,timestamp
2023-03-04,3040.020996,2023-03-04
2023-03-05,3686.450195,2023-03-05
2023-03-06,3312.134521,2023-03-06
2023-03-07,3389.550049,2023-03-07
2023-03-08,3294.054443,2023-03-08
2023-03-09,3184.903564,2023-03-09
2023-03-10,3063.218262,2023-03-10
2023-03-11,2919.337402,2023-03-11
2023-03-12,3798.683838,2023-03-12
2023-03-13,2729.974609,2023-03-13


In [ ]:
dt = [pred_TG_A_J, pred_TG_A_S, pred_TG_B_J, pred_TG_B_S, pred_TG_C_J, pred_TG_C_S, pred_TG_D_J, pred_TG_D_S, pred_TG_E_J, pred_TG_E_S,
    pred_CR_A_J, pred_CR_B_J, pred_CR_C_J, pred_CR_D_J, pred_CR_D_S, pred_CR_E_J, pred_CR_E_S,
    pred_CB_A_J, pred_CB_A_S, pred_CB_D_J, pred_CB_E_J,
    pred_RD_A_J, pred_RD_A_S, pred_RD_C_S, pred_RD_D_J, pred_RD_D_S, pred_RD_E_J, pred_RD_E_S,
    pred_BC_A_J, pred_BC_A_S, pred_BC_B_J, pred_BC_B_S, pred_BC_C_J, pred_BC_C_S, pred_BC_D_J, pred_BC_E_J, pred_BC_E_S,
    pred_CB_F_J, pred_RD_F_J]

new_names = [
    'TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S', 'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S',
    'CR_A_J', 'CR_B_J', 'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S',
    'CB_A_J', 'CB_A_S', 'CB_D_J', 'CB_E_J',
    'RD_A_J', 'RD_A_S', 'RD_C_S', 'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S',
    'BC_A_J', 'BC_A_S', 'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J', 'BC_E_S',
    'CB_F_J', 'RD_F_J'
]
# Assuming df1 and dt (list of predicted dataframes) are already defined

for i, common_part in enumerate(new_names):
    # Filter df1 to include only rows with the common part in ID_1
    filtered_df1 = df1[df1['ID_1'].str.endswith(common_part)]

    # Merge predicted dataframe and filtered_df1 based on timestamp and ID_2
    merged_df = pd.merge(dt[i], filtered_df1, left_on='timestamp', right_on='ID_2')

    # Update the 'answer' column in df1 with the 'price' column from predicted dataframe
    df1.loc[df1['ID_1'].str.endswith(common_part), 'answer'] = merged_df['predicted_price'].values


In [ ]:
df1['Day_of_week'] = df1['ID_2'].dt.day_name()

# 'Day_of_week'이 'Sunday'인 행의 '예측 price'를 0으로 설정
df1.loc[df1['Day_of_week'] == 'Sunday', 'answer'] = 0

# 'Day_of_week' 열을 삭제 (필요한 경우)
df1.drop('Day_of_week', axis=1, inplace=True)

In [ ]:
df1.drop('ID_1', axis=1, inplace=True)
df1.drop('ID_2', axis=1, inplace=True)

In [ ]:
df1

,ID,answer
0,TG_A_J_20230304,3040.020996
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3312.134521
3,TG_A_J_20230307,3389.550049
4,TG_A_J_20230308,3294.054443
...,...,...
1087,RD_F_J_20230327,465.105255
1088,RD_F_J_20230328,489.279236
1089,RD_F_J_20230329,476.339325
1090,RD_F_J_20230330,466.677277


In [ ]:
df1.to_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/nosunlstm.csv', index = False)